In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [2]:
df=pd.read_csv('editing.csv')

In [3]:
df.head()

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,required_car_parking_space,lead_time,arrival_year,arrival_month,arrival_date,repeated_guest,...,room_type_reserved_3,room_type_reserved_4,room_type_reserved_5,room_type_reserved_6,market_segment_type_0,market_segment_type_1,market_segment_type_2,market_segment_type_3,market_segment_type_4,has_children
0,2,0,1,4,0,118,2017,12,28,0,...,0,0,0,0,0,1,0,0,0,0
1,2,1,0,2,0,17,2018,4,14,0,...,0,0,0,0,0,1,0,0,0,1
2,1,0,1,5,0,349,2018,10,4,0,...,0,0,0,0,1,0,0,0,0,0
3,1,0,2,4,0,69,2018,6,12,0,...,0,0,0,0,1,0,0,0,0,0
4,2,0,0,4,0,11,2018,1,20,0,...,0,0,0,0,0,1,0,0,0,0


In [4]:
X=df.drop(columns=['booking_status'])
y=df['booking_status']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [6]:
y_train.shape ,y_test.shape

((13602,), (4535,))

In [7]:
NOcanceled =y_train.sum()
total=y_train.shape[0]
NOcanceled/total

0.3258344361123364

In [8]:
NOcanceled =y_test.sum()
total=y_test.shape[0]
NOcanceled/total

0.33296582138919517

Logistic Regression

In [9]:
log=LogisticRegression()
log.fit(X_train,y_train)

/Users/rahim/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [10]:
yd=log.predict(X_train)

In [11]:
accuracy_score(yd,y_train)


0.7959123658285546

In [12]:
yed=log.predict(X_test)

In [13]:
accuracy_score(yed,y_test)


0.802425578831312

Random Forest Classifier

In [14]:
rfc = RandomForestClassifier(n_estimators=100,max_depth=5)
rfc.fit(X_train, y_train)

RandomForestClassifier(max_depth=5)

In [15]:
y_pred = rfc.predict(X_train)

accuracy = accuracy_score(y_train, y_pred)
accuracy

0.7973092192324658

In [16]:
y_pred = rfc.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
accuracy

0.7953693495038588

In [17]:
rfc = RandomForestClassifier(n_estimators=100)
param_grid = {'max_depth': range(1, 31)}

grid_search = GridSearchCV(rfc, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

print("Best parameters: ", grid_search.best_params_)
print("Accuracy on testing data: ", grid_search.score(X_test, y_test))

Best parameters:  {'max_depth': 20}
Accuracy on testing data:  0.896361631753032


KNeighborsClassifie

In [20]:
knn = KNeighborsClassifier()
param_grid = {'n_neighbors': range(1, 31)}
grid_search = GridSearchCV(knn, param_grid, cv=10, scoring='accuracy')
grid_search.fit(X_train, y_train)

print("Best value of n_neighbors:", grid_search.best_params_)

accuracy = grid_search.score(X_test, y_test)
print("Accuracy of KNN classifier with best n_neighbors:", accuracy)


/Users/rahim/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/rahim/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer b

Best value of n_neighbors: {'n_neighbors': 2}
Accuracy of KNN classifier with best n_neighbors: 0.804851157662624


/Users/rahim/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


support vector machine (poly)

In [19]:
for i in range(2,10):
    svmc = SVC(kernel='poly',degree=i)
    svmc.fit(X_train, y_train)

    y_pred = svmc.predict(X_train)
    print(i)
    accuracy = accuracy_score(y_train, y_pred)
    print(accuracy)

    y_pred = svmc.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    print(accuracy)

2
0.7616526981326276
0.7618522601984564
3
0.7626084399353036
0.7614112458654906
4
0.7627554771357153
0.7614112458654906
5
0.7602558447287163
0.760529217199559
6
0.7591530657256286
0.7589856670341786
7
0.7561388031171886
0.7556780595369349
8
0.7534186149095721
0.7543550165380375
9
0.7526834289075136
0.7545755237045204


In [ ]:
#cross vlatidion 

#insamble 

# Feature Engering